In [1]:
!pip install nltk  
!pip install bert-score
!pip install rouge-score
!pip install peft
!pip install nltk  
!pip install bert-score
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.

In [2]:
!pip install nltk  
!pip install bert-score
!pip install rouge-score
!pip install peft
!pip install nltk  
!pip install bert-score
!pip install rouge-score

In [3]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, AutoModelForVisualQuestionAnswering
from sklearn.metrics import accuracy_score, f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import numpy as np
from PIL import Image
import logging
from tqdm import tqdm
import gc

2025-05-18 06:35:54.883142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747550155.084968      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747550155.150968      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# ========== CHECK CUDA/GPU ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"PyTorch Version: {torch.__version__}")
    print(f"CUDA Device Count: {torch.cuda.device_count()}")
    torch.cuda.empty_cache()
else:
    print("WARNING: CUDA not available, falling back to CPU. Check CUDA installation or GPU settings in Kaggle (Settings -> Accelerator -> GPU).")


Using device: cuda
GPU Name: Tesla T4
CUDA Version: 12.4
PyTorch Version: 2.6.0+cu124
CUDA Device Count: 2


In [5]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, AutoModelForVisualQuestionAnswering
from sklearn.metrics import accuracy_score, f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score
import numpy as np
from PIL import Image
import logging
from tqdm import tqdm
import gc

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ========== PARAMETERS ==========
base_model_name = "Salesforce/blip-vqa-base"
batch_size = 100
val_chunk_size = 10000  # Number of validation questions to process
output_csv = "/kaggle/working/vqa_baseline_results.csv"

# Paths
csv_base_path = '/kaggle/input/vqa-vr/'
img_base_path = '/kaggle/input/vqa-vr/filtered_images_corrected/'
val_csv_path = os.path.join(csv_base_path, 'val_main_image_id.csv')  # Adjust if using a different validation CSV
train_csv_path = os.path.join(csv_base_path, 'train_main_image_id.csv')  # Used if splitting train data

# Create output directory
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

# ========== CHECK CUDA/GPU ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")
if device.type == "cuda":
    logger.info(f"GPU Name: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

# Custom Dataset Class for VQA
class VQADataset(Dataset):
    def __init__(self, csv_path, img_base_path, processor, max_questions=None, split_from_train=False, split_ratio=0.1):
        """
        Initialize the dataset.
        - csv_path: Path to the CSV file.
        - img_base_path: Directory containing images.
        - processor: BLIP processor.
        - max_questions: Maximum number of questions to process.
        - split_from_train: If True, split validation data from train_csv_path.
        - split_ratio: Fraction of training data to use as validation if split_from_train is True.
        """
        if split_from_train:
            df = pd.read_csv(csv_path)
            # Randomly sample split_ratio of the training data for validation
            self.df = df.sample(frac=split_ratio, random_state=42)[['path', 'question', 'correct_answer']]
            logger.info(f"Created validation split: {len(self.df)} samples from training data")
        else:
            self.df = pd.read_csv(csv_path)[['path', 'question', 'correct_answer']]
        self.img_base_path = img_base_path
        self.processor = processor
        self.valid_indices = self._validate_dataset(max_questions)
        self.unique_images = len(set(self.df['path'].iloc[self.valid_indices]))
        logger.info(f"Loaded {len(self.valid_indices)} valid samples from {csv_path}")
        logger.info(f"Number of unique images: {self.unique_images}")

    def _validate_dataset(self, max_questions=None):
        valid_indices = []
        for idx in range(len(self.df)):
            if max_questions is not None and len(valid_indices) >= max_questions:
                break
            try:
                img_path = os.path.join(self.img_base_path, self.df['path'][idx])
                image = Image.open(img_path).convert('RGB')
                question = self.df['question'][idx]
                inputs = self.processor(image, question, return_tensors="pt", padding=True, truncation=True)
                if 'pixel_values' in inputs and inputs['pixel_values'] is not None:
                    valid_indices.append(idx)
            except Exception as e:
                logger.warning(f"Skipping index {idx}: Failed to load/process image {img_path}. Error: {str(e)}")
        return valid_indices

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        actual_idx = self.valid_indices[idx]
        img_path = os.path.join(self.img_base_path, self.df['path'][actual_idx])
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            logger.error(f"Failed to load image at index {actual_idx}: {img_path}. Error: {str(e)}")
            raise e
        question = self.df['question'][actual_idx]
        answer = str(self.df['correct_answer'][actual_idx]).lower()
        inputs = self.processor(image, question, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs['labels'] = self.processor.tokenizer.encode(answer, return_tensors="pt", padding=True, truncation=True).squeeze(0)
        return inputs

# Custom collate function
def custom_collate_fn(batch):
    keys = batch[0].keys()
    result = {}
    for key in keys:
        max_len = max(item[key].shape[-1] for item in batch)
        if key in ['input_ids', 'attention_mask', 'labels']:
            padded_tensors = torch.zeros(len(batch), max_len, dtype=torch.long)
            for i, item in enumerate(batch):
                tensor = item[key]
                padded_tensors[i, :len(tensor)] = tensor
            result[key] = padded_tensors
        elif key == 'pixel_values':
            result[key] = torch.stack([item[key] for item in batch])
        else:
            result[key] = torch.stack([item[key] for item in batch])
    return result

# Function to perform inference
def inference_and_save(model, processor, val_dataset, output_csv_path, batch_size):
    model.to(device)
    model.eval()
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=custom_collate_fn)
    results = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(val_loader, desc="Baseline Inference", unit="batch")):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            outputs = model.generate(**inputs)
            predicted_answers = [processor.decode(output, skip_special_tokens=True).lower() for output in outputs]
            batch_size_actual = len(predicted_answers)
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size_actual, len(val_dataset))
            batch_paths = [val_dataset.df['path'][val_dataset.valid_indices[i]] for i in range(start_idx, end_idx)]
            batch_questions = [val_dataset.df['question'][val_dataset.valid_indices[i]] for i in range(start_idx, end_idx)]
            batch_correct = [str(val_dataset.df['correct_answer'][val_dataset.valid_indices[i]]).lower() for i in range(start_idx, end_idx)]
            for path, question, correct, predicted in zip(batch_paths, batch_questions, batch_correct, predicted_answers):
                results.append({
                    'path': path,
                    'question': question,
                    'correct_answer': correct,
                    'predicted_answer': predicted
                })
    return results

# Function to compute metrics
def compute_metrics(results_df):
    logger.info("Computing baseline evaluation metrics...")
    metrics = {
        'accuracy': 0.0,
        'f1_score': 0.0,
        'bleu_score': 0.0,
        'rouge1': 0.0,
        'rougeL': 0.0,
        'bertscore_f1': 0.0
    }
    y_true = results_df['correct_answer'].tolist()
    y_pred = results_df['predicted_answer'].tolist()
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['f1_score'] = f1_score(y_true, y_pred, average='weighted')
    logger.info(f"Baseline Accuracy: {metrics['accuracy']:.4f}")
    logger.info(f"Baseline F1 Score: {metrics['f1_score']:.4f}")
    bleu_scores = []
    smoothie = SmoothingFunction().method1
    for true, pred in zip(y_true, y_pred):
        true_tokens = true.split()
        pred_tokens = pred.split()
        bleu = sentence_bleu([true_tokens], pred_tokens, smoothing_function=smoothie)
        bleu_scores.append(bleu)
    metrics['bleu_score'] = np.mean(bleu_scores)
    logger.info(f"Baseline Average BLEU Score: {metrics['bleu_score']:.4f}")
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rougeL_scores = []
    for true, pred in zip(y_true, y_pred):
        scores = rouge.score(true, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    metrics['rouge1'] = np.mean(rouge1_scores)
    metrics['rougeL'] = np.mean(rougeL_scores)
    logger.info(f"Baseline Average ROUGE-1 F1 Score: {metrics['rouge1']:.4f}")
    logger.info(f"Baseline Average ROUGE-L F1 Score: {metrics['rougeL']:.4f}")
    # Compute BERTScore
    try:
        P, R, F1 = score(y_pred, y_true, lang="en", verbose=True)
        metrics['bertscore_f1'] = F1.mean().item()
        logger.info(f"Baseline Average BERTScore F1: {metrics['bertscore_f1']:.4f}")
    except Exception as e:
        logger.error(f"Failed to compute BERTScore: {str(e)}")
        metrics['bertscore_f1'] = 0.0
    print("========== BASELINE EVALUATION RESULTS ==========")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"F1 Score: {metrics['f1_score']:.4f}")
    print(f"Average BLEU Score: {metrics['bleu_score']:.4f}")
    print(f"Average ROUGE-1 F1 Score: {metrics['rouge1']:.4f}")
    print(f"Average ROUGE-L F1 Score: {metrics['rougeL']:.4f}")
    print(f"Average BERTScore F1: {metrics['bertscore_f1']:.4f}")
    print("=================================================")
    return metrics

# Main function
def main():
    print("========== BASELINE EVALUATION STARTED ==========")
    try:
        # Load pre-trained model and processor
        logger.info("Loading pre-trained BLIP-VQA model...")
        model = AutoModelForVisualQuestionAnswering.from_pretrained(base_model_name)
        processor = AutoProcessor.from_pretrained(base_model_name)

        # Load validation dataset
        use_split = not os.path.exists(val_csv_path)
        if use_split:
            logger.info("No validation CSV found. Splitting 10% of training data for validation.")
            val_dataset = VQADataset(
                train_csv_path, img_base_path, processor,
                max_questions=val_chunk_size, split_from_train=True, split_ratio=0.1
            )
        else:
            logger.info(f"Using validation CSV: {val_csv_path}")
            val_dataset = VQADataset(val_csv_path, img_base_path, processor, max_questions=val_chunk_size)

        # Perform inference
        logger.info(f"Running baseline inference on {len(val_dataset)} validation samples...")
        val_results = inference_and_save(model, processor, val_dataset, output_csv, batch_size=batch_size)

        # Save results to CSV
        results_df = pd.DataFrame(val_results)
        results_df.to_csv(output_csv, index=False)
        logger.info(f"Baseline results saved to {output_csv}")

        # Compute metrics
        metrics = compute_metrics(results_df)

    finally:
        # Clean up
        torch.cuda.empty_cache()
        gc.collect()
        logger.info("GPU memory cleaned up")
        print("========== BASELINE EVALUATION ENDED ==========")

if __name__ == "__main__":
    main()

========== BASELINE EVALUATION STARTED ==========


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

calculating scores...
computing bert embedding.


  0%|          | 0/28 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/157 [00:00<?, ?it/s]

done in 4.17 seconds, 2400.90 sentences/sec
========== BASELINE EVALUATION RESULTS ==========
Accuracy: 0.2748
F1 Score: 0.2339
Average BLEU Score: 0.0498
Average ROUGE-1 F1 Score: 0.3056
Average ROUGE-L F1 Score: 0.3056
Average BERTScore F1: 0.9457
========== BASELINE EVALUATION ENDED ==========
